In [ ]:
import math
import torch
import re

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 54.8MB/s 
     |████████████████████████████████| 901kB 53.5MB/s 


In [ ]:
!unzip /content/drive/MyDrive/file.zip -d /content/sample_data

Archive:  /content/drive/MyDrive/file.zip
   creating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/trainer_state.json  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/merges.txt  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/vocab.json  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/eval_results.json  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/training_args.bin  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/tokenizer_config.json  
  inflating: /content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel/pytorch_model.bin  
  inflating: /content/sample_data/content/

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('/content/sample_data/content/transformers/examples/pytorch/language-modeling/HPmodel')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [ ]:
def ppl_score(generated, tokenizer, model):

    encodings = tokenizer(generated, return_tensors='pt')

    max_length = model.config.n_positions
    stride = 512

    lls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    print('\n')
    return ppl


In [ ]:
sentence = """Harry Potter and The Cursed Child. Autumn seemed to arrive suddenly that year. The
morning of the first of September was crisp and
golden as an apple, and as the little family bobbed
across the rumbling road toward the great sooty
station, the fumes of car exhausts and the breath of
pedestrians sparkled like cobwebs in the cold air. Two
large cages rattled on top of the laden trolleys the
parents were pushing; the owls inside them hooted
indignantly, and the redheaded girl trailed tearfully
behind her brothers, clutching her father’s arm.
“It won’t be long, and you’ll be going too,” Harry told
her.
“Two years,” sniffed Lily. “I want to go now\”
The commuters stared curiously at the owls as the
family wove its way toward the barrier between
platforms nine and ten. Albus’s voice drifted back to
Harry over the surrounding clamor; his sons had
resumed the argument they had started in the car.
“I won’ti I won’t be in Slytherin!”
“James, give it a rest!” said Ginny.
“I only said he might be,” said James, grinning at his
younger brother. “There’s nothing wrong with that. He
might be in Slyth — ”
But James caught his mother’s eye and fell silent.
The five Potters approached the barrier. With a
slightly cocky look over his shoulder at his younger
brother, James took the trolley from his mother and
broke into a run. A moment later, he had vanished.
“You’ll write to me, won’t you?” Albus asked his
parents immediately, capitalizing on the momentary
absence of his brother.
“Every day, if you want us to,” said Ginny.
“Not every day,” said Albus quickly. “James says most
people only get letters from home about once a
month.”
“We wrote to James three times a week last year,”
said Ginny.
“And you don’t want to believe everything he tells you
about Hogwarts,” Harry put in. “He likes a laugh,
your brother.”
Side by side, they pushed the second trolley forward,
gathering speed. As they reached the barrier, Albus
winced, but no collision came. Instead, the family
emerged onto platform nine and three-quarters,
which was obscured by thick white steam that was
pouring from the scarlet Hogwarts Express. Indistinct
figures were swarming through the mist, into which
James had already disappeared.
“Where are they?” asked Albus anxiously, peering at
the hazy forms they passed as they made their way
down the platform."""

sentence = re.sub(r'\s', ' ', sentence)
input_ids = tokenizer.encode(sentence, return_tensors='pt')
result = model.generate(
    input_ids, 
    max_length=1000,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

generated = tokenizer.decode(result[0], skip_special_tokens=True)
generated

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Harry Potter and The Cursed Child. Autumn seemed to arrive suddenly that year. The morning of the first of September was crisp and golden as an apple, and as the little family bobbed across the rumbling road toward the great sooty station, the fumes of car exhausts and the breath of pedestrians sparkled like cobwebs in the cold air. Two large cages rattled on top of the laden trolleys the parents were pushing; the owls inside them hooted indignantly, and the redheaded girl trailed tearfully behind her brothers, clutching her father’s arm. “It won’t be long, and you’ll be going too,” Harry told her. “Two years,” sniffed Lily. “I want to go now\\” The commuters stared curiously at the owls as the family wove its way toward the barrier between platforms nine and ten. Albus’s voice drifted back to Harry over the surrounding clamor; his sons had resumed the argument they had started in the car. “I won’ti I won’t be in Slytherin!” “James, give it a rest!” said Ginny. “I only said he might b

In [ ]:
print(ppl_score(generated, tokenizer, model))

100%|██████████| 2/2 [00:19<00:00,  9.64s/it]



tensor(23.9650)


In [ ]:
sentence = """When Harry walked in there was a sudden hush, and
then everybody started talking loudly at once. He
slipped into a seat between Ron and Hermione at the
Gryffindor table and tried to ignore the fact that
people were standing up to look at him.
Fortunately, Dumbledore arrived moments later. The
babble died away.
“Another year gone!” Dumbledore said cheerfully.
“And I must trouble you with an old man’s wheezing
waffle before we sink our teeth into our delicious
feast. What a year it has been! Hopefully your heads
are all a little fuller than they were . . . you have the
whole summer ahead to get them nice and empty
before next year starts. ...
“Now, as I understand it, the House Cup here needs
awarding, and the points stand thus: In fourth place,
Gryffindor, with three hundred and twelve points; in
third, Hufflepuff, with three hundred and fifty-two;
Ravenclaw has four hundred and twenty-six and
Slytherin, four hundred and seventy- two.”
A storm of cheering and stamping broke out from the
Slytherin table. Harry could see Draco Malfoy banging
his goblet on the table. It was a sickening sight.
“Yes, yes, well done, Slytherin,” said Dumbledore.
“However, recent events must be taken into account.”
The room went very still. The Slytherins’ smiles faded
a little.
“Ahem,” said Dumbledore. “I have a few last-minute
points to dish out. Let me see. Yes ...
“First — to Mr. Ronald Weasley ...”
Ron went purple in the face; he looked like a radish
with a bad sunburn.
"... for the best-played game of chess Hogwarts has
seen in many years, I award Gryffindor House fifty
points.”
Gryffindor cheers nearly raised the bewitched ceiling;
the stars overhead seemed to quiver. Percy could be
heard telling the other prefects, “My brother, you
know! My youngest brother! Got past McGonagall’s
giant chess set!”
At last there was silence again.
“Second — to Miss Hermione Granger . . . for the use of
cool logic in the face of fire, I award Gryffindor House
fifty points.”
Hermione buried her face in her arms; Harry strongly
suspected she had burst into tears. Gryffindors up
and down the table were beside themselves — they
were a hundred points up.
“Third — to Mr. Harry Potter ...” said Dumbledore.
The room went deadly quiet. "... for"""

sentence = re.sub(r'\s', ' ', sentence)
input_ids = tokenizer.encode(sentence, return_tensors='pt')
result = model.generate(
    input_ids, 
    max_length=1000,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

generated = tokenizer.decode(result[0], skip_special_tokens=True)
generated


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'When Harry walked in there was a sudden hush, and then everybody started talking loudly at once. He slipped into a seat between Ron and Hermione at the Gryffindor table and tried to ignore the fact that people were standing up to look at him. Fortunately, Dumbledore arrived moments later. The babble died away. “Another year gone!” Dumbledore said cheerfully. “And I must trouble you with an old man’s wheezing waffle before we sink our teeth into our delicious feast. What a year it has been! Hopefully your heads are all a little fuller than they were... you have the whole summer ahead to get them nice and empty before next year starts.... “Now, as I understand it, the House Cup here needs awarding, and the points stand thus: In fourth place, Gryffindor, with three hundred and twelve points; in third, Hufflepuff, with three hundred and fifty-two; Ravenclaw has four hundred and twenty-six and Slytherin, four hundred and seventy- two.” A storm of cheering and stamping broke out from the Sl

In [ ]:
print(ppl_score(generated, tokenizer, model))

100%|██████████| 2/2 [00:19<00:00,  9.56s/it]



tensor(14.2953)


In [ ]:
sentence = """The rest of the final term passed in a haze of blazing 
sunshine. Hogwarts was back to normal with only a 
few, small differences — Defense Against the Dark 
Arts classes were canceled (“but we’ve had plenty of 
practice at that anyway,” Ron told a disgruntled 
Hermione) and Lucius Malfoy had been sacked as a 
school governor. Draco was no longer strutting 
around the school as though he owned the place. On 
the contrary, he looked resentful and sulky. On the 
other hand, Ginny Weasley was perfectly happy 
again. 

Too soon, it was time for the journey home on the 
Hogwarts Express. Harry, Ron, Hermione, Fred, 
George, and Ginny got a compartment to themselves. 
They made the most of the last few hours in which 
they were allowed to do magic before the holidays. 
They played Exploding Snap, set off the very last of 
Fred and George’s Filibuster fireworks, and practiced 
disarming each other by magic. Harry was getting 
very good at it. 

They were almost at King’s Cross when Harry 
remembered something. 

“Ginny — what did you see Percy doing, that he didn’t 
want you to tell anyone?” 

“Oh, that,” said Ginny, giggling. “Well — Percy’s got a 
girlfriend.” 

Fred dropped a stack of books on George’s head. 
“What?” 

“It’s that Ravenclaw prefect, Penelope Clearwater,” 
said Ginny. “That’s who he was writing to all last 
summer. He’s been meeting her all over the school in 
secret. I walked in on them kissing in an empty """

sentence = re.sub(r'\s', ' ', sentence)
input_ids = tokenizer.encode(sentence, return_tensors='pt')
result = model.generate(
    input_ids, 
    max_length=1000,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

generated = tokenizer.decode(result[0], skip_special_tokens=True)
generated


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'The rest of the final term passed in a haze of blazing  sunshine. Hogwarts was back to normal with only a  few, small differences — Defense Against the Dark  Arts classes were canceled (“but we’ve had plenty of  practice at that anyway,” Ron told a disgruntled  Hermione) and Lucius Malfoy had been sacked as a  school governor. Draco was no longer strutting  around the school as though he owned the place. On  the contrary, he looked resentful and sulky. On the  other hand, Ginny Weasley was perfectly happy  again.   Too soon, it was time for the journey home on the  Hogwarts Express. Harry, Ron, Hermione, Fred,  George, and Ginny got a compartment to themselves.  They made the most of the last few hours in which  they were allowed to do magic before the holidays.  They played Exploding Snap, set off the very last of  Fred and George’s Filibuster fireworks, and practiced  disarming each other by magic. Harry was getting  very good at it.   They were almost at King’s Cross when Harry  re

In [ ]:
print(ppl_score(generated, tokenizer, model))

100%|██████████| 2/2 [00:20<00:00, 10.09s/it]



tensor(33.5475)


In [ ]:
sentence = """Harry turned to look outside. Something very small
and gray was bobbing in and out of sight beyond the
glass. He stood up for a better look and saw that it
was a tiny owl, carrying a letter that was much too
big for it. The owl was so small, in fact, that it kept
tumbling over in the air, buffeted this way and that in
the train’s slipstream. Harry quickly pulled down the
window, stretched out his arm, and caught it. It felt
like a very fluffy Snitch. He brought it carefully inside.
The owl dropped its letter onto Harry’s seat and began
zooming around their compartment, apparently very
pleased with itself for accomplishing its task. Hedwig
clicked her beak with a sort of dignified disapproval.
Crookshanks sat up in his seat, following the owl with
his great yellow eyes. Ron, noticing this, snatched the
owl safely out of harm’s way.
Harry picked up the letter. It was addressed to him.
He ripped open the letter, and shouted, “It’s from
Sirius!”
“What?” said Ron and Hermione excitedly. “Read it
aloud!”
Dear Harry,
I hope this finds you before you reach your aunt and
uncle. I don’t know whether they’re used to owl post
Buckbeak and I are in hiding. I won’t tell you where, in
case this owl falls into the wrong hands. I have some
doubt about his reliability, but he is the best I could
find, and he did seem eager for the job.
I believe """

sentence = re.sub(r'\s', ' ', sentence)
input_ids = tokenizer.encode(sentence, return_tensors='pt')
result = model.generate(
    input_ids, 
    max_length=1000,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

generated = tokenizer.decode(result[0], skip_special_tokens=True)
generated

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Harry turned to look outside. Something very small and gray was bobbing in and out of sight beyond the glass. He stood up for a better look and saw that it was a tiny owl, carrying a letter that was much too big for it. The owl was so small, in fact, that it kept tumbling over in the air, buffeted this way and that in the train’s slipstream. Harry quickly pulled down the window, stretched out his arm, and caught it. It felt like a very fluffy Snitch. He brought it carefully inside. The owl dropped its letter onto Harry’s seat and began zooming around their compartment, apparently very pleased with itself for accomplishing its task. Hedwig clicked her beak with a sort of dignified disapproval. Crookshanks sat up in his seat, following the owl with his great yellow eyes. Ron, noticing this, snatched the owl safely out of harm’s way. Harry picked up the letter. It was addressed to him. He ripped open the letter, and shouted, “It’s from Sirius!” “What?” said Ron and Hermione excitedly. “R

In [ ]:
print(ppl_score(generated, tokenizer, model))

100%|██████████| 2/2 [00:19<00:00,  9.71s/it]



tensor(29.4207)
